# Exploring Ebay Car Sales Data


I work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.
The dataset was originally scraped and uploaded to Kaggle (https://www.kaggle.com/orgesleka/used-cars-database/data).



# Introduction

The aim of this project is to clean the data and analyze the included used car listings.
The data dictionary provided with data is as follows:



- **dateCrawled** - When this ad was first crawled. All field-values are taken from this date.
- **name** - Name of the car.
- **seller** - Whether the seller is private or a dealer.
- **offerType** - The type of listing
- **price** - The price on the ad to sell the car.
- **abtest** - Whether the listing is included in an A/B test.
- **vehicleType** - The vehicle Type.
- **yearOfRegistration** - The year in which the car was first registered.
- **gearbox** - The transmission type.
- **powerPS** - The power of the car in PS.
- **model** - The car model name.
- **kilometer** - How many kilometers the car has driven.
- **monthOfRegistration** - The month in which the car was first registered.
- **fuelType** - What type of fuel the car uses.
- **brand** - The brand of the car.
- **notRepairedDamage** - If the car has a damage which is not yet repaired.
- **dateCreated** - The date on which the eBay listing was created.
- **nrOfPictures** - The number of pictures in the ad.
- **postalCode** - The postal code for the location of the vehicle.
- **lastSeenOnline** - When the crawler saw this ad last online.

In [1]:
import numpy as np
import pandas as pd


In [2]:
autos= pd.read_csv('autos.csv', encoding = 'latin-1')
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   dateCrawled          371528 non-null  object
 1   name                 371528 non-null  object
 2   seller               371528 non-null  object
 3   offerType            371528 non-null  object
 4   price                371528 non-null  int64 
 5   abtest               371528 non-null  object
 6   vehicleType          333659 non-null  object
 7   yearOfRegistration   371528 non-null  int64 
 8   gearbox              351319 non-null  object
 9   powerPS              371528 non-null  int64 
 10  model                351044 non-null  object
 11  kilometer            371528 non-null  int64 
 12  monthOfRegistration  371528 non-null  int64 
 13  fuelType             338142 non-null  object
 14  brand                371528 non-null  object
 15  notRepairedDamage    299468 non-nu

# Data Cleaning

From above, the following observations were made:

- The dataset contains 20 columns, most of which are strings.
- Some columns have null values, but none have more than ~20% null values.
- The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

In [4]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [5]:
autos.columns = [
       'date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'kilometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'
]

# Exploring the Kilometer and Price Columns

We'll start by analyzing the kilometer and price columns. Here's the steps we'll take:
Analyze the columns using minimum and maximum values and look for any values that look unrealistically high or low (outliers) that we might want to remove.
We'll use:
- Series.unique().shape to see how many unique values
- Series.describe() to view min/max/median/mean etc
- Series.value_counts(), with some variations:
- chained to .head() if there are lots of values.

Because Series.value_counts() returns a series, we can use Series.sort_index() with ascending= True or False to view the highest and lowest values with their counts (can also chain to head() here).
When removing outliers, we can do df[(df["col"] > x ) & (df["col"] < y )], but it's more readable to use df[df["col"].between(x,y)]

In [6]:
autos['kilometer'].unique().shape

(13,)

In [7]:
autos['kilometer'].describe

<bound method NDFrame.describe of 0         150000
1         125000
2         125000
3         150000
4          90000
           ...  
371523     20000
371524    125000
371525    150000
371526    150000
371527     50000
Name: kilometer, Length: 371528, dtype: int64>

In [8]:
autos['kilometer'].value_counts()

150000    240797
125000     38067
100000     15920
90000      12523
80000      11053
70000       9773
60000       8669
50000       7615
5000        7069
40000       6376
30000       6041
20000       5676
10000       1949
Name: kilometer, dtype: int64

In [9]:
autos['price'].unique().shape

(5597,)

In [10]:
autos['price'].describe

<bound method NDFrame.describe of 0           480
1         18300
2          9800
3          1500
4          3600
          ...  
371523     2200
371524     1199
371525     9200
371526     3400
371527    28990
Name: price, Length: 371528, dtype: int64>

In [11]:
autos['price'].value_counts().head()

0       10778
500      5670
1500     5394
1000     4649
1200     4594
Name: price, dtype: int64

# Exploring the date columns
Let's now move on to the date columns and understand the date range the data covers.
There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:
- **dataCrawled**: added by the crawler
- **lastSeen**: added by the crawler
- **ad_created**: from the website
- **registration_month**: from the website
- **registration_year**: from the website

Right now, the date_crawled, last_seen, and ad_created columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively. The other two columns are represented as numeric values, so we can use methods like Series.describe() to understand the distribution without any extra data processing.
Let's first understand how the values in the three string columns are formatted. These columns all represent full timestamp values, like so:

In [12]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-24 11:52:17,2016-03-24 00:00:00,2016-04-07 03:16:57
1,2016-03-24 10:58:45,2016-03-24 00:00:00,2016-04-07 01:46:50
2,2016-03-14 12:52:21,2016-03-14 00:00:00,2016-04-05 12:47:46
3,2016-03-17 16:54:04,2016-03-17 00:00:00,2016-03-17 17:40:17
4,2016-03-31 17:25:20,2016-03-31 00:00:00,2016-04-06 10:17:21


In [13]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-24 11:52:17,2016-03-24 00:00:00,2016-04-07 03:16:57
1,2016-03-24 10:58:45,2016-03-24 00:00:00,2016-04-07 01:46:50
2,2016-03-14 12:52:21,2016-03-14 00:00:00,2016-04-05 12:47:46
3,2016-03-17 16:54:04,2016-03-17 00:00:00,2016-03-17 17:40:17
4,2016-03-31 17:25:20,2016-03-31 00:00:00,2016-04-06 10:17:21


In [14]:
autos['date_crawled'].value_counts(normalize=True, dropna=False)

2016-03-24 14:49:47    0.000019
2016-03-19 21:49:56    0.000016
2016-03-26 22:57:31    0.000016
2016-03-07 17:36:19    0.000013
2016-03-11 15:36:59    0.000013
                         ...   
2016-03-19 14:55:03    0.000003
2016-03-24 17:52:30    0.000003
2016-04-04 17:48:06    0.000003
2016-03-23 15:52:32    0.000003
2016-04-03 04:32:43    0.000003
Name: date_crawled, Length: 280500, dtype: float64

In [15]:
autos['date_crawled'].sort_index()

0         2016-03-24 11:52:17
1         2016-03-24 10:58:45
2         2016-03-14 12:52:21
3         2016-03-17 16:54:04
4         2016-03-31 17:25:20
                 ...         
371523    2016-03-14 17:48:27
371524    2016-03-05 19:56:21
371525    2016-03-19 18:57:12
371526    2016-03-20 19:41:08
371527    2016-03-07 19:39:19
Name: date_crawled, Length: 371528, dtype: object

In [16]:
autos['ad_created'].value_counts(normalize=True, dropna=False)

2016-04-03 00:00:00    0.038893
2016-04-04 00:00:00    0.037741
2016-03-20 00:00:00    0.036463
2016-03-12 00:00:00    0.036011
2016-03-21 00:00:00    0.035812
                         ...   
2015-08-10 00:00:00    0.000003
2015-03-20 00:00:00    0.000003
2016-01-14 00:00:00    0.000003
2016-01-08 00:00:00    0.000003
2015-12-17 00:00:00    0.000003
Name: ad_created, Length: 114, dtype: float64

In [17]:
autos['ad_created'].sort_index()

0         2016-03-24 00:00:00
1         2016-03-24 00:00:00
2         2016-03-14 00:00:00
3         2016-03-17 00:00:00
4         2016-03-31 00:00:00
                 ...         
371523    2016-03-14 00:00:00
371524    2016-03-05 00:00:00
371525    2016-03-19 00:00:00
371526    2016-03-20 00:00:00
371527    2016-03-07 00:00:00
Name: ad_created, Length: 371528, dtype: object

In [18]:
autos['last_seen'].value_counts(normalize=True, dropna=False)

2016-04-07 06:45:59    0.000046
2016-04-06 13:45:54    0.000046
2016-04-07 07:44:31    0.000043
2016-04-06 09:17:58    0.000043
2016-04-06 10:15:26    0.000043
                         ...   
2016-03-11 10:16:37    0.000003
2016-03-25 19:31:13    0.000003
2016-03-25 10:16:14    0.000003
2016-03-30 03:46:30    0.000003
2016-03-24 12:42:58    0.000003
Name: last_seen, Length: 182806, dtype: float64

In [19]:
autos['last_seen'].sort_index()

0         2016-04-07 03:16:57
1         2016-04-07 01:46:50
2         2016-04-05 12:47:46
3         2016-03-17 17:40:17
4         2016-04-06 10:17:21
                 ...         
371523    2016-04-06 00:46:52
371524    2016-03-11 18:17:12
371525    2016-04-07 07:15:26
371526    2016-03-24 12:45:21
371527    2016-03-22 03:17:10
Name: last_seen, Length: 371528, dtype: object

In [20]:
autos['registration_year'].describe()

count    371528.000000
mean       2004.577997
std          92.866598
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64

# Dealing with Incorrect Registration Year Data
One thing that stands out from the exploration we did in the last screen is that the registration_year column contains some odd values:
- The minimum value is 1000, before cars were invented
- The maximum value is 9999, many years into the future

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.
Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [21]:
autos.loc[~ autos['registration_year'].between(1899, 2017), 'registration_year'].shape

(4202,)

In [22]:
autos.loc[~ autos['registration_year'].between(1899, 2017), 'registration_year'] = np.nan

In [23]:
autos.dropna(subset=['registration_year'], axis=0, inplace=True)

In [24]:
autos['registration_year'].value_counts(normalize=True)

2000.0    0.066837
1999.0    0.061980
2005.0    0.060753
2006.0    0.055074
2001.0    0.055041
            ...   
1925.0    0.000003
1920.0    0.000003
1919.0    0.000003
1915.0    0.000003
1911.0    0.000003
Name: registration_year, Length: 98, dtype: float64

In [25]:
autos.shape

(367326, 20)

* Exploring Price by Brand
When working with data on cars, it's natural to explore variations across different car brands. We can use aggregation to understand the brand column. Here's what the process looks like:
- Identify the unique values we want to aggregate by
- Create an empty dictionary to store our aggregate data
- Loop over the unique values, and for each loop I will (1) subset the dataframe by the unique values, (2) calculate the mean of whichever column we're interested in and, lastly (3) ssign the val/mean to the dict as k/v.

In [26]:
autos['brand'].unique()

array(['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'seat', 'citroen',
       'honda', 'fiat', 'opel', 'mini', 'smart', 'hyundai',
       'sonstige_autos', 'alfa_romeo', 'subaru', 'volvo', 'mitsubishi',
       'kia', 'suzuki', 'lancia', 'porsche', 'toyota', 'chevrolet',
       'dacia', 'daihatsu', 'trabant', 'saab', 'chrysler', 'jaguar',
       'daewoo', 'rover', 'land_rover', 'lada'], dtype=object)

In [27]:
autos['brand'].value_counts()

volkswagen        78572
bmw               39955
opel              39624
mercedes_benz     35016
audi              32633
ford              25275
renault           17648
peugeot           10913
fiat               9530
seat               6899
mazda              5628
skoda              5594
smart              5191
citroen            5129
nissan             4987
toyota             4657
sonstige_autos     3893
hyundai            3605
mini               3352
volvo              3309
mitsubishi         3034
honda              2797
kia                2527
alfa_romeo         2323
suzuki             2307
porsche            2205
chevrolet          1816
chrysler           1437
dacia               889
jeep                799
daihatsu            796
subaru              772
land_rover          765
jaguar              619
trabant             586
daewoo              537
saab                525
rover               485
lancia              478
lada                219
Name: brand, dtype: int64

In [28]:
brand_mean_prices = {}
brands = autos['brand'].unique()

for brand in brands:
    brand_mean_prices[brand] = autos.loc[autos['brand'] == brand, 'price'].mean()

In [29]:
brand_mean_prices

{'volkswagen': 14677.971389298988,
 'audi': 15947.492722091134,
 'jeep': 12042.379224030037,
 'skoda': 6424.702717196997,
 'bmw': 14590.780277812539,
 'peugeot': 3177.0065976358474,
 'ford': 8527.891552917903,
 'mazda': 5713.083333333333,
 'nissan': 4543.245638660517,
 'renault': 2342.912454669084,
 'mercedes_benz': 17338.188885081105,
 'seat': 4379.713291781418,
 'citroen': 8936.418990056542,
 'honda': 3811.91455130497,
 'fiat': 5369.487932843652,
 'opel': 3233.977008883505,
 'mini': 9895.394391408114,
 'smart': 3546.3007127721057,
 'hyundai': 5426.885991678225,
 'sonstige_autos': 627054.5861803236,
 'alfa_romeo': 36041.2677572105,
 'subaru': 4229.926165803109,
 'volvo': 5077.751888788153,
 'mitsubishi': 3278.6502966381017,
 'kia': 5677.150771666007,
 'suzuki': 4380.435197225835,
 'lancia': 3175.3723849372386,
 'porsche': 51162.89070294784,
 'toyota': 5237.571827356667,
 'chevrolet': 7476.205396475771,
 'dacia': 5847.721034870641,
 'daihatsu': 1696.6557788944724,
 'trabant': 20577.237

# Storing Aggregate Data in a DataFrame
We aggregated across brands to understand mean price. We observed that in the top 6 brands, there's a distinct price gap.
- Audi, BMW and Mercedes Benz are more expensive
- Ford and Opel are less expensive
- Volkswagen is in between

For the top 6 brands, let's use aggregation to understand the average mileage for those cars and if there's any visible link with mean price. While our natural instinct may be to display both aggregated series objects and visually compare them, this has a few limitations:
- it's difficult to compare more than two aggregate series objects if we want to extend to more columns
- we can't compare more than a few rows from each series object
- we can only sort by the index (brand name) of both series objects so we can easily make visual comparisons

Instead, we can combine the data from both series objects into a single dataframe (with a shared index) and display the dataframe directly. To do this, we'll need to learn two pandas methods:
- pandas series constructor
- pandas dataframe constructor

In [33]:
brand_mean_kilometer = {}
for brand in brands:
    brand_mean_kilometer[brand] = autos.loc[autos['brand'] == brand, 'kilometer'].mean()

In [34]:
brand_mean_kilometer

{'volkswagen': 128561.9559130479,
 'audi': 129674.4093402384,
 'jeep': 120913.6420525657,
 'skoda': 113835.35931355023,
 'bmw': 132785.0081341509,
 'peugeot': 124861.63291487217,
 'ford': 123813.25420375865,
 'mazda': 126013.68159203981,
 'nissan': 119733.3065971526,
 'renault': 128073.15276518586,
 'mercedes_benz': 130734.66415352981,
 'seat': 121334.97608349036,
 'citroen': 120514.72021836616,
 'honda': 125947.4436896675,
 'fiat': 116817.94333683106,
 'opel': 128908.7421764587,
 'mini': 94331.74224343675,
 'smart': 100377.57657484108,
 'hyundai': 104638.00277392511,
 'sonstige_autos': 87035.70511173902,
 'alfa_romeo': 128861.38613861386,
 'subaru': 126528.49740932643,
 'volvo': 138115.74493804775,
 'mitsubishi': 127102.83454185893,
 'kia': 109491.49188761377,
 'suzuki': 106991.76419592544,
 'lancia': 124466.52719665272,
 'porsche': 98396.8253968254,
 'toyota': 117403.90809534035,
 'chevrolet': 100154.18502202643,
 'dacia': 86237.34533183352,
 'daihatsu': 119007.53768844221,
 'trabant

In [35]:
bmp = pd.Series(brand_mean_prices)
bmk = pd.Series(brand_mean_kilometer)
bm = pd.DataFrame(bmp, columns=['mean_prices'])
bm['mean_kilometer'] = bmk

In [36]:
bm

,mean_prices,mean_kilometer
volkswagen,14677.971389,128561.955913
audi,15947.492722,129674.409340
jeep,12042.379224,120913.642053
skoda,6424.702717,113835.359314
bmw,14590.780278,132785.008134
peugeot,3177.006598,124861.632915
ford,8527.891553,123813.254204
mazda,5713.083333,126013.681592
nissan,4543.245639,119733.306597
renault,2342.912455,128073.152765
